In [1]:
import math 
import pandas as pd
import random 
import numpy as np
import pickle
import itertools

In [4]:
# Setting simulation parameters values

# N = [30, 50, 100, 500, 1000]  # population size
# D = [0.01, 0.5, 1, 2] # Densities
# T = [1, 2, 5, 7, 10] # infectious time in days

N = [30]  # population size
D = [0.01] # Densities
T = [1, 2] # infectious time in days


d = 2 # distance in meters 
iter = 100 # number of iterations
time = 1440 # exposure time in minutes


In [3]:
# import best model's trace 

with open(r"D:\Manal\Programming\project1\R0_simulation\trace_enveloppe_2m_0.1.pkl", 'rb') as f:
    trace_env = pickle.load(f)


p0_sample = trace_env.posterior["p0"].stack(sample=("chain", "draw")).values
p0_sample = np.random.choice(p0_sample, size=100)


lambda_sample = trace_env.posterior["lambda"].stack(sample=("chain", "draw")).values
lambda_sample = np.random.choice(lambda_sample, size=100)

In [ ]:
%%time 

# Perform the simulations

data = []
current_row = None

for k in D: # density
    for n in N: # population size 
        for t in T:
            for i in range(iter): # iterations
                p0 = p0_sample[i]
                lambda_ = lambda_sample[i]
                results = 0
                
                for m in range(n): # individuals
                    mu = np.random.uniform(0, 1) # a random number between 0 and 1
                    Rmax= np.sqrt(n/(k * np.pi)) # maximum radius distance
                    r0 = np.sqrt((n * mu)/(k * np.pi)) # generate initial radius distance r0
                    r = r0
                    p = 1 - (p0 * np.exp(-lambda_ * r0)) 
                    proba_minute = p
                    for j in range(time):
                        if j == 0:
                            pass
                        else:
                            theta_ =  np.random.uniform(0, 2*np.pi) # a random number between 0 and pi
                            r = min(np.sqrt(r**2 + d**2 - 2 * r * d * np.cos(theta_) ),Rmax) # generate next radius distance per minute 
                            p = 1 - (p0 * np.exp(-lambda_ * r)) # probability per minute
                            proba_minute *= p # product on all probabilities per minute for each ID
                            
                    proba_T = 1 - proba_minute ** t # apply the exponent on infectious time 
                    status = np.random.binomial(1, proba_T) # Bernoulli drawing for infectious status (0 or 1) using the final proba
                    results += status # sum on all Positives 

                # Check if the combination of parameters changes or still the same : to append number of positives as colomns instead of rows

                new_combination = current_row is None or (n, k, t) != (current_row["size"], current_row["density"],  current_row["day"])

                if new_combination:
                    # Create a new row
                    current_row = {"size": n, "density": k,  "day": t}
                    data.append(current_row)

                # Appending the new values of results as an additional columns
                current_row[f"sim{i}"] = str(results)
                    
print("Done")

In [13]:
%%time 

# Perform the simulations

data = []

for k in D: # density
    for n in N: # population size 
            for i in range(iter): # iterations
                p0 = p0_sample[i]
                lambda_ = lambda_sample[i]
                temp_results = []
                results = []
                for m in range(n): # individuals
                    mu = np.random.uniform(0, 1) # a random number between 0 and 1
                    Rmax= np.sqrt(n/(k * np.pi)) # maximum radius distance
                    r0 = np.sqrt((n * mu)/(k * np.pi)) # generate initial radius distance r0
                    r = r0
                    p = 1 - (p0 * np.exp(-lambda_ * r0)) 
                    proba_minute = p
                    
                    for j in range(time):
                        if j == 0:
                            pass
                        else:
                            theta_ =  np.random.uniform(0, 2*np.pi) # a random number between 0 and pi
                            r = min(np.sqrt(r**2 + d**2 - 2 * r * d * np.cos(theta_) ),Rmax) # generate next radius distance per minute 
                            p = 1 - (p0 * np.exp(-lambda_ * r)) # probability per minute
                            proba_minute *= p # product on all probabilities per minute for each ID
                    
                    temp_results.append(proba_minute)

                    for t in T:
                        proba_T = [1 - val**t for val in temp_results] # apply the exponent on infectious time
                        status = [np.random.binomial(1, val) for val in proba_T] # Bernoulli drawing for infectious status (0 or 1) using the final proba
                        results = sum(status)

CPU times: total: 21.7 s
Wall time: 22.5 s


In [ ]:
# Convert to dataframe for easy view
df = pd.DataFrame(data)

# Show the header (10 first lines)
df.head(10)

In [ ]:
# Save the simulation results into a csv file

df.to_csv("simulation_results_v4.csv", index=False)